Carga del primer Dataset, conteniendo titulos con su respectiva información.
Se procede con la limpieza inicial, eliminación de columnas sin información y desanidamiento de columnas compuestas.

In [82]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/movies_dataset.csv")

C:\Users\Gaston Luna Paez\AppData\Local\Temp\ipykernel_13656\2834309086.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/movies_dataset.csv")


Descarte de filas sin release_date para su posterior manipulación

In [83]:
df['release_date'] = df['release_date'].apply(lambda x: pd.to_datetime(x, errors='coerce').date()) # Con coerce marcamos con nan aquellos errores, de modo de descartarlos luego
dfNAN = df[df.release_date.isna()]
df.dropna(subset=["release_date"], inplace=True)

In [84]:
df['release_date'] = pd.to_datetime(df['release_date'])

In [85]:
df['release_year'] = df.release_date.apply(lambda x: x.year) #Creación de columna con el año de las fechas de lanzamiento

Eliminación de columnas innecesarias para los pasos posteriores

In [86]:
df.drop(columns=['video', 'imdb_id', 'adult','original_title', 'poster_path', 'homepage'], inplace=True)

Imputación de valores faltantes para las columnas budget y revenue, necesarios para la creación de la columna return

In [87]:
#Se encontro valores de tipo str en la columna 'budget', debían ser casteados para rellenarse con 0
df['budget'] = df['budget'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(value=0)

In [88]:
#Casteo de valores a float para creación de nueva columna 'return'
df[['revenue', 'budget']] = df[['revenue', 'budget']].astype(float)
df['return'] = df['revenue']/df['budget']
df['return'] = df['return'].fillna(value=0)
df['return'].replace([np.inf, -np.inf], 0, inplace=True) # Salvando divisiones por 0


A cotinuación se observa que muchos tipos de datos no estan como deberían en cuanto a tipo. Sin embargo, por simpleza, estos casteos se realizaran posterior al etl en cuestion

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4488 non-null   object        
 1   budget                 45376 non-null  float64       
 2   genres                 45376 non-null  object        
 3   id                     45376 non-null  object        
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  object        
 7   production_companies   45376 non-null  object        
 8   production_countries   45376 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45376 non-null  float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       45376 non-null  object        
 13  status

DESANIDAMIENTO DE CAMPOS

Campo: 'belongs_to_collection'

Tipo de dato y formato inicial:

<class 'str'>
{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}

En este campo y en los posteriores anidados se observo la presencia de estructuras similares a codigo python, por ende se empleo la función literal_eval(str) que interpreta las cadenas como expresiones, a diferencia del eval() esta intrpreta solo expresiones seguras 

In [90]:
from ast import literal_eval

In [91]:
string = "{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"
literal_eval(string)

for elemento in df['belongs_to_collection']:
    if pd.notnull(elemento):
        if not(isinstance(eval(elemento), dict)):
            print(elemento)

In [92]:
df['belongs_to_collection'] = df['belongs_to_collection'].apply( lambda x: literal_eval(x) if pd.notnull(x) else x)
df['belongs_to_collection'] = df['belongs_to_collection'].apply( lambda x: x['name'] if isinstance(x,dict) else '') #Se aplica un valor imputado aquellos registros sin collection asociada, no se usa palabras evitando confusiones por coincidencias accidentales

In [93]:
df['belongs_to_collection']

0                  Toy Story Collection
1                                      
2             Grumpy Old Men Collection
3                                      
4        Father of the Bride Collection
                      ...              
45460                                  
45462                                  
45463                                  
45464                                  
45465                                  
Name: belongs_to_collection, Length: 45376, dtype: object

DESANIDAMIENTO DE CAMPOS

Campo: 'genres'

Tipo de dato y formato inicial:

<class 'str'>

"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"

En este caso se observa una lista de diccionarios, se opto por quedarnos con los nombres de los generos para mejor interpretación

In [94]:
lista = literal_eval(df.iloc[0].genres) # Ya tenia forma de lista de diccionarios
[genero['name'] for genero in lista] #Me quedo solo con los nombres de los generos, podrian ser los id tambien

['Animation', 'Comedy', 'Family']

In [95]:
df['genres'] = df['genres'].apply( lambda x: literal_eval(x) if pd.notnull(x) else list([]))
df['genres'] = df['genres'].apply( lambda x: [genero['name'] for genero in x] )

In [96]:
df['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45460        [Drama, Action, Romance]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45376, dtype: object

DESANIDAMIENTO DE CAMPOS

Campo: 'production_companies'

Tipo de dato y formato inicial:

<class 'str'>

"[{'name': 'Pixar Animation Studios', 'id': 3}]"


In [97]:
string = "[{'name': 'Pixar Animation Studios', 'id': 3}]"
print(literal_eval(string))

for elemento in df['production_companies']:
    if pd.notnull(elemento):
        if not(isinstance(literal_eval(elemento), list)):
            print(elemento)

[{'name': 'Pixar Animation Studios', 'id': 3}]


In [98]:
df['production_companies'] = df['production_companies'].apply( lambda x: literal_eval(x) if pd.notnull(x) else x)
df['production_companies'] = df['production_companies'].apply( lambda x: [prod['name'] for prod in x if type(prod) == dict] if isinstance(x, list) else list([]))

In [99]:
df.production_companies

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45460    [Westdeutscher Rundfunk (WDR), Working Title F...
45462                                        [Sine Olivia]
45463                            [American World Pictures]
45464                                          [Yermoliev]
45465                                                   []
Name: production_companies, Length: 45376, dtype: object

DESANIDAMIENTO DE CAMPOS

Campo: 'production_countries'

Tipo de dato y formato inicial:

<class 'str'>

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"



In [100]:
string = "[{'iso_3166_1': 'US', 'name': 'United States of America'}]"
print(literal_eval(string))

for elemento in df['production_countries']:
    if pd.notnull(elemento):
        if not(isinstance(eval(elemento), list)):
            print(elemento)

[{'iso_3166_1': 'US', 'name': 'United States of America'}]


In [101]:
df['production_countries'] = df['production_countries'].apply( lambda x: literal_eval(x) if pd.notnull(x) else x )
df['production_countries'] = df['production_countries'].apply( lambda x: [country['name'] for country in x] if isinstance(x,list) else list([]))

In [102]:
df.production_countries

0                               [United States of America]
1                               [United States of America]
2                               [United States of America]
3                               [United States of America]
4                               [United States of America]
                               ...                        
45460    [Canada, Germany, United Kingdom, United State...
45462                                        [Philippines]
45463                           [United States of America]
45464                                             [Russia]
45465                                     [United Kingdom]
Name: production_countries, Length: 45376, dtype: object

DESANIDAMIENTO DE CAMPOS

Campo: 'spoken_languages'

Tipo de dato y formato inicial:

<class 'str'>

"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]"

En este caso, dada la prescencia de Paises escritos en otros idiomas, empleando caracteres especiales, como persas por ejemplos, se obtienen los formatos iso

In [103]:
string = "[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]"
print(literal_eval(string))

for elemento in df['spoken_languages']:
    if pd.notnull(elemento):
        if not(isinstance(literal_eval(elemento), list)):
            print(elemento)

[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]


In [104]:
df['spoken_languages'] = df['spoken_languages'].apply( lambda x: literal_eval(x) if pd.notnull(x) else x )
df['spoken_languages'] = df['spoken_languages'].apply( lambda x: [lang['iso_639_1'] for lang in x] if isinstance(x,list) else list([]))

In [105]:
df.spoken_languages

0            [en]
1        [en, fr]
2            [en]
3            [en]
4            [en]
           ...   
45460        [en]
45462        [tl]
45463        [en]
45464          []
45465        [en]
Name: spoken_languages, Length: 45376, dtype: object

Por ultimo y para simplicidad posterior del uso de los datos, principalmente aquellos que se uitilizan como entradas, se paso a minusculas todos los titulos de las peliculas, posteriormente se realizo lo mismo con los actores y directores asociados.

In [106]:
df['title'] = df['title'].apply(lambda x: x.title() if pd.notnull(x) else x)

In [107]:
df.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[en],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[en, fr]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[en],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000
3,,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[en],Released,Friends are the people who let you be yourself...,Waiting To Exhale,6.1,34.0,1995,5.090760
4,Father of the Bride Collection,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,[en],Released,Just When His World Is Back To Normal... He's ...,Father Of The Bride Part Ii,5.7,173.0,1995,0.000000


ELIMINACION DE DUPLICADOS

Se analizo el dataset, donde se observo una triada de un registro duplicado y varios duplicados en pareja.

In [108]:
df['id'].describe()

count      45376
unique     45346
top       141971
freq           3
Name: id, dtype: object

In [109]:
df.id.isna().sum()

0

In [110]:
# df = df.sort_values('id')
# df = df[ ~df.duplicated(subset=['revenue', 'runtime', 'release_date', 'tagline', 'title','overview' ], keep='first')] # De esta forma me quedo con los que poseen id, dados que estan primeros en el orden

In [111]:
df = df[ ~df.duplicated(subset=['id'], keep='first')]

In [112]:
df['id'].describe()

count     45346
unique    45346
top         862
freq          1
Name: id, dtype: object

TRABAJANDO CON EL DATASET DE CREDITS

En este caso necesitamos para el trabajo posterior la lista de actores y su director y el id correspondiente a la pelicula.

In [113]:
df2 = pd.read_csv("data/credits.csv")
df2.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


DESANIDAMIENTO DE CAMPOS

Campo: 'cast'

Tipo de dato y formato inicial:

<class 'str'>

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg'} ....


In [114]:
df2['cast'] = df2['cast'].apply( lambda x: [ actor['name'].title() for actor in literal_eval(x) ] if pd.notnull(x) else list([]) )


In [115]:
type(df2['cast'].iloc[0])

list

DESANIDAMIENTO DE CAMPOS

Campo: 'crew'

Tipo de dato y formato inicial:

<class 'str'>


'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'},..."

In [116]:
#Me tengo que quedar con el director --> \'job\': \'Director\', \'name\':
# Existen casos con más de un director, por eso lista
df2['director'] = df2['crew'].apply( lambda x: [ data['name'].title() for data in eval(x) if data['job'] == 'Director' ] if pd.notnull(x) else list([]))

In [117]:
df2.drop(columns = ['crew'], inplace = True)

ELIMINACION DE DUPLICADOS

Se verifica con el describe y el value_count para agrupar los duplicados. Se verifica con la lista de indices en algunos casos para posteriormente eliminarlos.

Cabe mencionar que en este dataset no se encontaron campos nulos

In [118]:
df2.isna().sum()

cast        0
id          0
director    0
dtype: int64

In [119]:
df2['id'].astype(str).describe()

count      45476
unique     45432
top       141971
freq           3
Name: id, dtype: object

In [120]:
df2['id'].astype(str).value_counts()

id
141971    3
298721    2
9755      2
10991     2
99080     2
         ..
55135     1
15877     1
72272     1
1549      1
461257    1
Name: count, Length: 45432, dtype: int64

In [121]:
df2[df2.id == 141971] #Confirmación de duplicidad

,cast,id,director
13261,"[Petteri Summanen, Ismo Kallio, Eppu Salminen,...",141971,[Jp Siili]
13375,"[Petteri Summanen, Ismo Kallio, Eppu Salminen,...",141971,[Jp Siili]
16764,"[Petteri Summanen, Ismo Kallio, Eppu Salminen,...",141971,[Jp Siili]


In [122]:
df2 = df2[~df2.id.duplicated(keep='first')] # Eliminación manteniendo la primer aparición, el caracter ~ es negación

In [123]:
df2.id.astype(str).describe()

count     45432
unique    45432
top         862
freq          1
Name: id, dtype: object

In [124]:
df2.columns

Index(['cast', 'id', 'director'], dtype='object')

ANTES DE GUARDAR LOS DATASET GENERADOS, SE PROCEDE A COMPARA UN REGISTRO CRUZANDO LA DATA DE AMBOS, TENIENDO EN CUENTA ESTO PARA REALIZAR UN MERGE Y QUEDARNOS CON LA DATA YA CRUZADA EN UN NUEVO DATASET GENERAL, AGILIZANDO DE ESTA MANERA LA RESPUESTA POSTERIOR DE LA API

In [125]:
df2[ df2['id'] == 5894]

,cast,id,director
153,"[Harvey Keitel, Lou Reed, Michael J. Fox, Rose...",5894,"[Paul Auster, Wayne Wang]"


In [126]:
df[df['id'] == '5894']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
153,Brooklyn Cigar Store Collection,2000000.0,[Comedy],5894,en,"Auggie runs a small tobacco shop in Brooklyn, ...",11.528147,"[Miramax Films, InterAL]",[United States of America],1995-09-15,1275000.0,83.0,[en],Released,Welcome to the planet Brooklyn.,Blue In The Face,6.8,28.0,1995,0.6375


Dada incongruencias en los tipos de datos, es necesario castear unas featuers, para lograr congruencia y a la vez compatibilidad entre los df para que el cruce de los datos sea viables

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45346 non-null  object        
 1   budget                 45346 non-null  float64       
 2   genres                 45346 non-null  object        
 3   id                     45346 non-null  object        
 4   original_language      45335 non-null  object        
 5   overview               44405 non-null  object        
 6   popularity             45346 non-null  object        
 7   production_companies   45346 non-null  object        
 8   production_countries   45346 non-null  object        
 9   release_date           45346 non-null  datetime64[ns]
 10  revenue                45346 non-null  float64       
 11  runtime                45100 non-null  float64       
 12  spoken_languages       45346 non-null  object        
 13  status

In [128]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45432 non-null  object
 1   id        45432 non-null  int64 
 2   director  45432 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [129]:
df.popularity = df.popularity.astype(float)

In [130]:
df.id = df.id.astype(int)

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45346 non-null  object        
 1   budget                 45346 non-null  float64       
 2   genres                 45346 non-null  object        
 3   id                     45346 non-null  int32         
 4   original_language      45335 non-null  object        
 5   overview               44405 non-null  object        
 6   popularity             45346 non-null  float64       
 7   production_companies   45346 non-null  object        
 8   production_countries   45346 non-null  object        
 9   release_date           45346 non-null  datetime64[ns]
 10  revenue                45346 non-null  float64       
 11  runtime                45100 non-null  float64       
 12  spoken_languages       45346 non-null  object        
 13  status

In [132]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cast      45432 non-null  object
 1   id        45432 non-null  int64 
 2   director  45432 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [133]:
df.to_csv('data/cleanMovies.csv', index=False, date_format="%Y-%m-%d")
df2.to_csv('data/cleanCredits.csv', index=False)

In [134]:
dfFinal = pd.merge(df, df2, on='id', how='inner')
dfFinal.shape

(45345, 22)

In [135]:
dfFinal.to_csv('data/cleanMergedData.csv', index=False, date_format="%Y-%m-%d")

Se tomo la decisión de descartar los datos sin data_release dado que uno de los endpoints del proyecto solicita trabajar con dicho campo, por ende evitando salidas erroneas se deja el tratamiento de dichos datos de manera más especifica para otra etapa del proyecto, por ejemplo, la incorporación posterior de más datos. Sin embargo dichos datos se trataran y se imputaran, marcandolos como corresponde; de manera tal de tenerlos en cuenta, si se solicita para los otros endpoints u pasos posteriores que pudiesen surgir. 

Además, cabe mencionar que se encontraron 6 filas con una carateristica distinta, donde se observa el desfase de unos datos, analizando en profundidad se encontro que son dos filas completas separadas en dos, ocasionando disturbios en los procesamientos. Sin embargo, dado el desfase, estas fueron extraidas junto con las filas sin release_date.
Las filas en cuestión, a modo de mencion, son las siguientes.

[19729, 29502, 35586] --> primeras partes de los datos

[19730, 29503, 35587] --> segunda parte de los datos anteriores pero ubicadas en las primeras columnas

Si bien estos datos no se trataron se muestran a modo de tener en cuenta a futuro en las areas correspondientes de la maniupulación y curación de datos.

In [136]:
dfNAN.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [137]:
dfNAN[dfNAN.index.isin([19729 , 29502, 35586])][['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview']]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview
19729,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,82663,tt0113002,en,Midnight Man,British soldiers force a recently captured IRA...
29502,False,"{'id': 122661, 'name': 'Mardock Scramble Colle...",0,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",http://m-scramble.jp/exhaust/,122662,tt2423504,ja,マルドゥック・スクランブル 排気,Third film of the Mardock Scramble series.
35586,False,NaN,0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...",NaN,249260,tt2622826,en,Avalanche Sharks,A group of skiers are terrorized during spring...


In [138]:
dfNAN[dfNAN.index.isin([19729 , 29502, 35586])]['overview'].iloc[2]

'A group of skiers are terrorized during spring break by some sharks that swim through the snow.'

In [139]:
dfNAN[dfNAN.index.isin([19730 , 29503, 35587])]['adult'].iloc[2]

' Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.'

In [140]:
dfNAN[dfNAN.index.isin([19730 , 29503, 35587])][['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries']]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,False,6.0
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,False,7.0
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,False,4.3
